# Visualize conditional dependence results
Load the saved toy dataset and plot how the contamination activates for different contextual values Z.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

data = np.load('data/toy_data.npz')
x, y, z = data['X'], data['Y'], data['Z']

plt.figure(figsize=(6, 4))
plt.scatter(z, y, s=4, alpha=0.3, label='Y vs Z')
plt.xlabel('Context Z')
plt.ylabel('Shear Y')
plt.title('Conditional activation of contamination by context')
plt.legend()
plt.tight_layout()
plt.show()
